# Vietnam Coastlines Combined


* Load stack of all available Landsat 5, 7, 8 and 9 satellite imagery for a location 
* Convert each satellite image into a remote sensing water index (MNDWI)
* For each satellite image, model ocean tides into a grid based on exact time of image acquisition
* Interpolate tide heights into spatial extent of image stack using the [FES2014 global tide model](https://github.com/GeoscienceAustralia/dea-coastlines/wiki/Setting-up-tidal-models-for-DEA-Coastlines)
* Mask out high and low tide pixels by removing all observations acquired outside of 50 percent of the observed tidal range centered over mean sea level
* Combine tidally-masked data into annual median composites representing the most representative position of the coastline at approximately mean sea level each year
* Apply morphological extraction algorithms to mask annual median composite rasters to a valid coastal region
* Extract waterline vectors using subpixel waterline extraction ([Bishop-Taylor et al. 2019b](https://doi.org/10.3390/rs11242984))
* Compute rates of coastal change at every 30 m using linear regression

This is an interactive version of the code intended for prototyping; to run this analysis at scale, use the [command line tools](DEACoastlines_generation_CLI.ipynb).

---

## Getting started

Set working directory to top level of repository to ensure links work correctly:

In [ ]:
cd ..

### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["USE_PYGEOS"] = "0"

# Load DEA Coastlines and DEA tools code
from coastlines.utils import get_study_site_geometry, load_config
from coastlines.combined import (
    load_and_mask_data_with_stac,
    export_results,
    filter_by_tides,
    generate_yearly_composites,
    mask_pixels_by_tide,
    sanitise_tile_id,
    mask_pixels_by_hillshadow,
    CoastlinesException,
)
from coastlines.vector import (
    contours_preprocess,
    contour_certainty,
    points_certainty,
)
from coastlines.continental import (
    generate_hotspots,
)

# Load other libraries
from pathlib import Path
import geopandas as gpd
from datacube.utils.dask import start_local_dask
from dea_tools.spatial import subpixel_contours
from odc.stac import configure_s3_access

In [ ]:
# Hide warnings. Don't run this cell to see the warnings.
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Create local dask client for parallelisation
dask_client = start_local_dask(
    n_workers=4, threads_per_worker=8, mem_safety_margin="2GB"
)

# Configure S3 access including request payer
_ = configure_s3_access(requester_pays=True, cloud_defaults=True)

print(
    dask_client.dashboard_link.replace(
        "/user", "https://hub.asia.easi-eo.solutions/user"
    )
)

## Setup


### Set analysis parameters

In [ ]:
# Study area selection
# study_area = "13,45" # North
# study_area = "9,19"  # South west
# study_area = "18,32" # Central

# Issues!
# study_area = "24,36"  # Too much noise still

study_area = "33,22"  # Missing half the island

# Config
version = "testing"
config_path = "configs/vietnam_coastlines_config_development.yaml"
config = load_config(config_path)

# Tide data and config
home = Path("~")
tide_data_location = f"{home}/tide_models"

# Output config
output_dir = Path(
    f"{config.output.location.rstrip('/')}/{version}/{sanitise_tile_id(study_area)}_{version}"
)
output_dir.mkdir(exist_ok=True, parents=True)
output_cache_zarr = (
    output_dir / f"{sanitise_tile_id(study_area)}_{version}_combined_ds.zarr"
)

# Load the geometry from the grid used for the location
geometry = get_study_site_geometry(config.input.grid_path, study_area)

# BBOX and other query parameters
bbox = list(
    geometry.to_crs(config.output.crs)
    .buffer(config.options.load_buffer_distance)
    .to_crs("EPSG:4326")
    .bounds.values[0]
)

# Use the USGS STAC API to identify scenes to load
query = {
    "bbox": bbox,
    "datetime": (str(config.options.start_year - 1), str(config.options.end_year + 1)),
}

In [ ]:
# View the AoI
geometry.explore(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
)

## Loading data

### Create spatiotemporal query using a STAC API as the backend
This establishes the spatial and temporal extent used to search for Landsat satellite data.


In [ ]:
# Load the data, mask it and generate composites.
# This is lazy-loaded, so no data is actually loaded yet.
ds, items = load_and_mask_data_with_stac(
    config, query, include_nir=True, optimise_combined=True
)

print(f"Found {len(items)} items")

ds

## Tidal modelling


### Interpolate tides into each satellite timestep
For each satellite timestep, model tide heights into a low-resolution 5 x 5 km grid (matching resolution of the FES2014 tidal model), then reproject modelled tides into the spatial extent of our satellite image. Add  this new data as a new variable in our satellite dataset to allow each satellite pixel to be analysed and filtered/masked based on the tide height at the exact moment of satellite image acquisition. 

In [ ]:
filtered = filter_by_tides(ds, tide_data_location, tide_centre)

print(
    f"Dropped {len(ds.time) - len(filtered.time)} out of {len(ds.time)} timesteps due to extreme tides"
)

## Generate yearly composites in memory
Export tidally-masked MNDWI median composites for each year, and three-yearly composites used to gapfill poor data coverage areas.

In [ ]:
# Optionally load the daily dataset into memory, either do this here or
# down below for the combined dataset. This takes more memory, but is
# faster. The below one results in a big, complex dask graph, but saves
# a fair bit of memory.
filtered = filtered.compute()
filtered

## Specific masking

First we do a per-pixel mask of the extreme tide pixels. This is done with data loaded into memory, to keep things efficient on memory and dask graphs.

The graph below shows the percentage of data remaining each year after per-pixel masking.

Second we do per-pixel hillshadow masking.

### Per-pixel tide masking

In [ ]:
# Per-pixel tide masking
pixel_tide_masked = mask_pixels_by_tide(filtered, tide_data_location, tide_centre)

In [ ]:
# Plot the percentage of pixels remaining after high-res tide masking
filter_null_byyear = (
    filtered[water_index].isnull().groupby("time.year").sum(dim=["time", "x", "y"])
)
masked_null_byyear = (
    pixel_tide_masked[water_index]
    .isnull()
    .groupby("time.year")
    .sum(dim=["time", "x", "y"])
)
diff = (filter_null_byyear / masked_null_byyear) * 100
diff.plot.line(x="year", ylim=[0, 100], figsize=(12, 4), yticks=range(0, 101, 20))

### Per-pixel hillshade masking

(Optional)

In [ ]:
# Run a hillshade masking step. This is optional, but can help remove
# some noisy pixels from the analysis.
try:
    pixel_tide_masked, hillshade = mask_pixels_by_hillshadow(
        pixel_tide_masked, items, debug=True
    )
except CoastlinesException as e:
    print("No DEM available for this area. Skipping hillshade masking.")

In [ ]:
# In the below plot, anything that isn't a `0` has had some
# pixels masked out due to hillshade.
mean_hs = hillshade.mean(dim="time")
mean_hs.plot.imshow(size=10, vmin=0, vmax=1)

In [ ]:
# Plot the percentage of pixels remaining after hillshade masking
shaded_null_byyear = (
    pixel_tide_masked[water_index]
    .isnull()
    .groupby("time.year")
    .sum(dim=["time", "x", "y"])
)
diff = (masked_null_byyear / shaded_null_byyear) * 100
diff.plot.line(x="year", ylim=[0, 100], figsize=(12, 4), yticks=range(0, 101, 20))

## Create annual composites

In [ ]:
# Create a yearly dataset, loaded into memory. This takes a long time!
combined_ds = generate_yearly_composites(
    pixel_tide_masked, start_year, end_year, water_index=water_index, include_nir=True
)

# Load the combined dataset instead. Make sure you comment out the filtered
# load step. This will take longer, but use less memory.
# combined_ds = combined_ds.compute()
combined_ds

In [ ]:
# Uncomment the below to save data

# if output_cache_zarr.exists():
#     import shutil

#     print(f"Folder {output_cache_zarr} already exists. Deleting...")
#     shutil.rmtree(output_cache_zarr)

# combined_ds.to_zarr(output_cache_zarr)

In [ ]:
# Uncomment the below to load data

# combined_ds = xr.open_zarr(output_cache_zarr).load()
# combined_ds

## Load vector data and pre-process raster data

In [ ]:
# Coastal mask modifications with "add" and "remove" areas
modifications_gdf = gpd.read_file(
    "/home/ubuntu/dea-coastlines/data/raw/vietnam_modifications.geojson", bbox=bbox
).to_crs(str(combined_ds.odc.crs))

# Mask dataset to focus on coastal zone only
(
    masked_ds,
    certainty_masks,
    all_time_20,
    all_time_80,
    river_mask,
    ocean_da,
    thresholded_ds,
    temporal_mask,
    annual_mask,
    coastal_mask,
    ocean_mask,
) = contours_preprocess(
    combined_ds=combined_ds,
    water_index=water_index,
    index_threshold=index_threshold,
    buffer_pixels=33,
    modifications_gdf=modifications_gdf,
    include_nir=True,
    debug=True,
)

# Plot a single timestep
masked_ds.isel(year=0).plot.imshow(size=8, cmap="RdBu_r")

In [ ]:
ocean_da.plot.imshow(size=8, cmap="RdBu")

In [ ]:
# Extract shorelines
contours = subpixel_contours(
    da=masked_ds,
    z_values=index_threshold,
    min_vertices=10,
    dim="year",
).set_index("year")

# Add quality measures
contours_with_certainty = contour_certainty(contours, certainty_masks)

In [ ]:
# Plot shorelines
contours_with_certainty.reset_index().explore(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    column="year",
    cmap="magma",
)

## Compute statistics

### Measure annual coastline movements

In [ ]:
from coastlines.combined import extract_points_with_movements

points = extract_points_with_movements(
    combined_ds, contours_with_certainty, baseline_year, start_year, end_year
)

In [ ]:
geomorphology_url = config.input.geomorphology_path
geomorphology_gdf = gpd.read_file(
    geomorphology_url,
    mask=geometry,
)

# And add certainty
points_with_certainty = points_certainty(
    points,
    geomorphology_gdf=geomorphology_gdf,
    baseline_year=baseline_year,
    rocky_query="(Preds == 'Bedrock') and (Probs > 0.75)",
)

In [ ]:
# Generate hotspots
hotspots = generate_hotspots(
    contours_with_certainty, points_with_certainty, [10000, 5000, 1000], baseline_year
)

# Visualise the 10_000 m hotspots
hotspots[0].explore(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    column="rate_time",
    cmap="RdYlGn",
    marker_kwds={
        "radius": 10,
        "fill_opacity": 0.8,
    },
)

## Export files

In [ ]:
clipped_points_gdf = points_with_certainty.clip(
    geometry.to_crs(points_with_certainty.crs)
)
clipped_contours_gdf = contours_with_certainty.clip(
    geometry.to_crs(contours_with_certainty.crs)
)

out_files = export_results(
    clipped_points_gdf, clipped_contours_gdf, version, str(output_dir), study_area
)

for out_file in out_files:
    print(f"Exported {out_file}")

### Close Dask client

In [ ]:
dask_client.close()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984